In [2]:
import requests, time, csv, os
import random
from datetime import datetime, timedelta
import cloudinary
import cloudinary.uploader
from googletrans import Translator

In [3]:
translator = Translator()

translate_cache = {}

def translate_vi_to_en(text):
    if not text:
        return text
    if text in translate_cache:
        return translate_cache[text]
    try:
        result = translator.translate(text, src="vi", dest="en")
        translate_cache[text] = result.text
        time.sleep(0.15)
        return result.text
    except:
        return text

cloudinary.config(
  cloud_name = "dvxmaiofh", 
  api_key = "834668983718514", 
  api_secret = "KI_mVAdKhFNvtbJN4w9TWSzJqno",
  secure = True
)

CATEGORY_ID = 53540
MAX_PRODUCTS = 4000

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

items = []
page = 1
MAX_PAGE = 150

def random_date(start_year=2022, end_year=2025):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    return (start + timedelta(
        days=random.randint(0, (end - start).days)
    )).strftime("%Y-%m-%d")

FAKE_MATERIALS = [
    "ABS Plastic",
    "Alloy",
    "Stainless Steel",
    "Silicone",
    "Synthetic Fabric"
]

FAKE_SIZES = ["S", "M", "L"]

FAKE_COLORS = [
    "Red",
    "Blue",
    "Green",
    "Yellow",
    "Orange",
    "Pink",
    "Purple",
    "Black",
    "White",
    "Gray",
    "Brown"
]

In [ ]:
print(f"Collecting products from Tiki category ID: {CATEGORY_ID}")

while len(items) < MAX_PRODUCTS and page <= MAX_PAGE:
    LIST_API = f"https://tiki.vn/api/v2/products?category={CATEGORY_ID}&page={page}&limit=40"
    resp = requests.get(LIST_API, headers=headers).json()
    data = resp.get("data", [])

    if not data:
        print(f"[WARN] Page {page} has no data, skipping...")
        page += 1
        continue

    for p in data:
        if len(items) >= MAX_PRODUCTS:
            break

        pid = p["id"]
        detail = requests.get(
            f"https://tiki.vn/api/v2/products/{pid}",
            headers=headers
        ).json()

        # ========== BASIC ==========
        product_name = detail.get("name")
        price = detail.get("price")
        stock = random.randint(50, 200)
        type_ = "accessory"

        # ========== IMAGES ==========
        imgList = [
            img["base_url"]
            for img in detail.get("images", [])
            if "base_url" in img
        ][:3]

        if not imgList:
            imgList = ["https://dummyimage.com/600x600"]

        manufacture_date = random_date(2022, 2024)
        entry_date = random_date(2024, 2025)
        expiry_date = random_date(2026, 2028)

        material = random.choice(FAKE_MATERIALS)
        size = random.choice(FAKE_SIZES)
        color = random.choice(FAKE_COLORS)

        cloudinary_urls = []
        print(f"   >>> Đang upload {len(imgList)} ảnh lên Cloudinary cho SP: {pid}...")
        
        for img_url in imgList:
            if img_url:
                try:
                    upload_result = cloudinary.uploader.upload(img_url, folder="PuppyLobby")
                    cloudinary_urls.append(upload_result['secure_url'])
                except Exception as e:
                    print(f"      [Lỗi Upload] Không thể up ảnh {img_url}: {e}")
                    cloudinary_urls.append(img_url)

        items.append({
            "product_name": translate_vi_to_en(product_name),
            "price": price,
            "manufacture_date": manufacture_date,
            "entry_date": entry_date,
            "expiry_date": expiry_date,
            "stock": stock,
            "type": type_,
            "imgList": cloudinary_urls,
            "size": size,
            "color": color,
            "material": material,
        })

        print(f"GET {len(items)} | {product_name[:40]}")
        time.sleep(0.35)

    page += 1
    time.sleep(0.5)

def sql_escape(value):
    if value is None:
        return "NULL"
    return "'" + str(value).replace("'", "''") + "'"

with open("insert_accessory_2.sql", "w", encoding="utf-8") as f:
    for item in items:
        f.write("-- ================================\n")
        f.write(f"-- {item['product_name']}\n")

        # ===== WITH new_product =====
        f.write("WITH new_product AS (\n")
        f.write("  INSERT INTO product (\n")
        f.write("    product_name, price, manufacture_date, entry_date, expiry_date, stock, type\n")
        f.write("  ) VALUES (\n")
        f.write(f"    {sql_escape(item['product_name'])},\n")
        f.write(f"    {item['price']},\n")
        f.write(f"    {sql_escape(item['manufacture_date'])},\n")
        f.write(f"    {sql_escape(item['entry_date'])},\n")
        f.write(f"    {sql_escape(item['expiry_date'])},\n")
        f.write(f"    {item['stock']},\n")
        f.write(f"    {sql_escape(item['type'])}\n")
        f.write("  ) RETURNING product_id\n")
        f.write(")")

        # ===== productimage CTEs =====
        for idx, img in enumerate(item["imgList"]):
            f.write(",\n")
            f.write(f"img_{idx} AS (\n")
            f.write(
                "  INSERT INTO productimage (product_id, image_url)\n"
                f"  SELECT product_id, {sql_escape(img)} FROM new_product\n"
                ")\n"
            )

        # ===== accessory (FINAL STATEMENT) =====
        f.write(
            "INSERT INTO accessory (product_id, size, color, material)\n"
            "SELECT product_id, "
            f"{sql_escape(item['size'])}, "
            f"{sql_escape(item['color'])}, "
            f"{sql_escape(item['material'])} "
            "FROM new_product;\n\n"
        )

   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 278020661...
